# BioLink Compliance Service

In the first phase of Translator, the knowledge graph standards working group created a standard schema for data transfer between reasoners, and a standard for representing graphs called the BioLink model. The knowledge graph portion of the reasoner schema defines the graph required to answer a query. For practical reasons, this schema is a more liberal superset of the BioLink Model schema.

The [BioLink compliance service](https://monarch-sandbox.cgrb.oregonstate.edu/docs) validates the knowledge graph portion of a ReasonerAPI message. Users can submit either a message object, or the knowledge graph portion of the Reasoner API standard.

### Basic Use

This tutorial assumes that you have the ability to generate Reasoner Standard compliant JSON.  [Another validator](http://transltr.io:7071/apidocs/) exists to validate against this schema.  This JSON can be posted to the BioLink compliance service.  The service will return 1 of 4 error codes:

422 - The JSON could not be parsed.  This occurs if the JSON object is not a valid Reasoner API response object  
418 - The JSON is not BioLink model compliant  
400 - The posted data is invalid JSON  
200 - The JSON was successfully validated  

In [42]:
import requests
import json
import os

compliance_url = 'https://monarch-sandbox.cgrb.oregonstate.edu/validate/knowledge_graph'

# First let's look at the JSON object we are sending the service
input_fh = open(os.path.abspath('') + '/resources/invalid.json', 'r')
input_data = json.load(input_fh)
print(json.dumps(input_data, indent=1))

{
 "edges": [
  {
   "id": "553903",
   "source_id": "https://omim.org/entry/603903",
   "target_id": "https://www.uniprot.org/uniprot/P00738",
   "relation_label": [
    "affects"
   ],
   "additionalProp1": {}
  }
 ],
 "nodes": [
  {
   "id": "OMIM:603903",
   "name": "Haptoglobin",
   "type": "Disease",
   "additionalProp1": {}
  }
 ],
 "additionalProp1": {}
}


In [43]:
response = requests.post(compliance_url, json=input_data)
print(json.dumps(response.json(), indent=1))

[
 {
  "entity": "https://www.uniprot.org/uniprot/P00738",
  "error_type": "MISSING_NODE_PROPERTY",
  "message": "Required node property 'category' missing",
  "message_level": "ERROR"
 },
 {
  "entity": "https://www.uniprot.org/uniprot/P00738",
  "error_type": "MISSING_NODE_PROPERTY",
  "message": "Required node property 'id' missing",
  "message_level": "ERROR"
 },
 {
  "entity": "https://www.uniprot.org/uniprot/P00738",
  "error_type": "MISSING_NODE_PROPERTY",
  "message": "Required node property 'name' missing",
  "message_level": "ERROR"
 },
 {
  "entity": "https://www.uniprot.org/uniprot/P00738",
  "error_type": "INVALID_CATEGORY",
  "message": "Node does not have a 'category' property",
  "message_level": "ERROR"
 },
 {
  "entity": "https://omim.org/entry/603903-https://www.uniprot.org/uniprot/P00738",
  "error_type": "MISSING_EDGE_PROPERTY",
  "message": "Required edge property 'relation' missing",
  "message_level": "ERROR"
 },
 {
  "entity": "https://omim.org/entry/603903-htt

Here we see there are 7 errors associated with the input object, going through them:

Errors 1-4 (MISSING_NODE_PROPERTY and INVALID_CATEGORY) are related to a missing node in the graph for the Haptoglobin protein.  We can resolve this by creating and adding it to the node list:

```
{
  "id": "https://www.uniprot.org/uniprot/P00738",
  "name": "Haptoglobin",
  "category": "Protein",
}

```

The next error, MISSING_EDGE_PROPERTY, is because we're missing the property 'relation' on our edge:

```
{
  "id": "553903",
  "source_id": "https://omim.org/entry/603903",
  "target_id": "https://www.uniprot.org/uniprot/P00738",
  "relation_label": [
      "affects"
  ],
  "relation": "affects",
}

```

The final two errors, INVALID_EDGE_PROPERTY_VALUE, are because we are using an IRI for the id property instead of a curie.
The get the curie mapping we can look at the biolink model jsonld context file here: https://github.com/biolink/biolink-model/blob/master/context.jsonld  

Here we see the two mappings:  
```
  "UniProtKB": "http://identifiers.org/uniprot/",
  "OMIM": "http://purl.obolibrary.org/obo/OMIM_",
```

Replacing the two IRIs with curies and putting it all together:  

```
{
  "edges": [
    {
      "id": "553903",
      "source_id": "OMIM:603903",
      "target_id": "UniProtKB:P00738",
      "relation_label": [
        "affects"
      ],
      "relation": "affects",
      "additionalProp1": {}
    }
  ],
  "nodes": [
    {
      "id": "OMIM:603903",
      "name": "Haptoglobin",
      "type": "Disease",
      "category": "Disease",
      "additionalProp1": {}
    },
    {
      "id": "UniProtKB:P00738",
      "name": "Haptoglobin",
      "category": "Protein",
      "additionalProp1": {}
    }
  ],
  "additionalProp1": {}
}
```

In [45]:
# Running the above object through the validator

input_fh = open(os.path.abspath('') + '/resources/fixed.json', 'r')
input_data = json.load(input_fh)
response = requests.post(compliance_url, json=input_data)
print(json.dumps(response.json(), indent=1))

{
 "message": "Successfully validated"
}
